In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [3]:
#Criando uma session com acesso ao gcp
spark = (
    SparkSession
    .builder
    .appName("Desafio-edc-format-trusted")
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
    .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    .config("fs.gs.auth.service.account.enable","true")
    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

21/09/12 22:54:33 WARN Utils: Your hostname, eduney-pc resolves to a loopback address: 127.0.1.1; using 172.26.68.160 instead (on interface eth0)
21/09/12 22:54:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/12 22:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/12 22:54:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Formatação dos dados de CNAE - Atividade Economica
Ajustar os tipos de dados para melhor leitura/joins

In [4]:
df_cnae = spark.read.parquet("gs://bootcamp-edc/raw/cnae.parquet/*.parquet")

In [5]:
df_cnae.limit(5).toPandas()

,cod_atividade,desc_atividade
0,0111301,Cultivo de arroz
1,0111302,Cultivo de milho
2,0111303,Cultivo de trigo
3,0111399,Cultivo de outros cereais não especificados an...
4,0112101,Cultivo de algodão herbáceo


In [6]:
df_cnae = (
    df_cnae
    .withColumn("cod_atividade", f.col("cod_atividade").cast("int"))
    .withColumn("desc_atividade", f.trim(f.col("desc_atividade")))
    )

In [7]:
df_cnae.limit(10).toPandas()

,cod_atividade,desc_atividade
0,111301,Cultivo de arroz
1,111302,Cultivo de milho
2,111303,Cultivo de trigo
3,111399,Cultivo de outros cereais não especificados an...
4,112101,Cultivo de algodão herbáceo
5,112102,Cultivo de juta
6,112199,Cultivo de outras fibras de lavoura temporária...
7,113000,Cultivo de cana-de-açúcar
8,114800,Cultivo de fumo
9,115600,Cultivo de soja


In [ ]:
df_cnae.printSchema()

In [9]:
df_cnae.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/cnae.parquet")

In [10]:
del(df_cnae)

## Leitura dos dados de Município

In [12]:
df_municipio = spark.read.parquet("gs://bootcamp-edc/raw/municipio.parquet/*.parquet")

In [13]:
df_municipio.limit(5).toPandas()

,cod_municipio,desc_municipio
0,0001,GUAJARA-MIRIM
1,0002,ALTO ALEGRE DOS PARECIS
2,0003,PORTO VELHO
3,0004,BURITIS
4,0005,JI-PARANA


In [16]:
df_municipio = (
    df_municipio
    .withColumn("cod_municipio", f.col("cod_municipio").cast("int"))
    .withColumn("desc_municipio", f.initcap(f.trim(f.col("desc_municipio"))))
)

In [22]:
df_municipio.limit(10).toPandas()

NameError: name 'df_municipio' is not defined

In [ ]:
df_municipio.printSchema()

In [18]:
df_municipio.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/municipio.parquet")

In [19]:
del(df_municipio)

## Leitura dos dados de estabelecimentos

In [20]:
df_estabelecimento = spark.read.parquet("gs://bootcamp-edc/raw/estabelecimento.parquet/*.parquet")

In [21]:
df_estabelecimento.limit(5).toPandas()

21/09/12 23:02:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial
0,31131846,0001,94,1,None,02,20201127,00,None,None,...,6969,11,23456677,None,None,None,None,None,None,None
1,31131858,0001,19,1,DANIBELEZA,02,20180726,00,None,None,...,7535,41,30166873,None,None,None,None,FERRARI@FERRARIASSESSORIA.COM.BR,None,None
2,31131946,0001,10,1,RUI OLIVEIRA,02,20210514,00,None,None,...,3849,71,85090334,None,None,None,None,RUI2017OLIVEIRA@HOTMAIL.COM,None,None
3,31132035,0001,08,1,MERCADINHO WIKANE,02,20180807,00,None,None,...,1417,88,99694036,None,None,None,None,MFRANCIENENUNES@GMAIL.COM,None,None
4,31132092,0001,97,1,None,02,20180807,00,None,None,...,7107,11,20212271,None,None,None,None,CONTATO@ZENITECONSULTORIA.COM.BR,None,None


In [24]:
#Formata colunas inteiras
int_columns = ["idc_matriz_filial","cod_situacao_cadastral","motivo_situacao_cadastral","cod_pais","cod_municipio"]
for column in int_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column, f.col(column).cast("int"))

In [25]:
#Formata colunas strings
str_columns = ["nome_fantasia","nome_cidade_exterior"]
for column in str_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column, f.initcap(f.trim(f.col(column))))
    
df_estabelecimento = df_estabelecimento.withColumn("email", f.lower(f.trim(f.col("email"))))

In [36]:
df_estabelecimento.limit(10).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,cod_municipio,ddd_telefone_1,num_telefone_1,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial
0,31131846,0001,94,1,None,2,20201127,0,None,NaN,...,6969,11,23456677,None,None,None,None,None,None,None
1,31131858,0001,19,1,Danibeleza,2,20180726,0,None,NaN,...,7535,41,30166873,None,None,None,None,ferrari@ferrariassessoria.com.br,None,None
2,31131946,0001,10,1,Rui Oliveira,2,20210514,0,None,NaN,...,3849,71,85090334,None,None,None,None,rui2017oliveira@hotmail.com,None,None
3,31132035,0001,08,1,Mercadinho Wikane,2,20180807,0,None,NaN,...,1417,88,99694036,None,None,None,None,mfrancienenunes@gmail.com,None,None
4,31132092,0001,97,1,None,2,20180807,0,None,NaN,...,7107,11,20212271,None,None,None,None,contato@zeniteconsultoria.com.br,None,None
5,31132149,0001,58,1,None,8,20181122,1,None,NaN,...,2521,74,91413324,None,None,None,None,vanusanaia016@gmail.com,None,None
6,31132209,0001,32,1,None,8,20181017,1,None,NaN,...,6639,19,34452693,None,None,None,None,ariana@sofisc.com.br,None,None
7,31132353,0001,79,1,Bruchez Prestadora De Servicos,8,20190116,1,None,NaN,...,89,00,00000000,None,None,None,None,None,None,None
8,31132395,0001,00,1,None,2,20180807,0,None,NaN,...,8501,55,96724760,None,None,None,None,None,None,None
9,13950538,0002,90,2,Espaco Da Moda,8,20190222,1,None,NaN,...,8055,47,91799760,None,None,None,None,None,None,None


In [35]:
df_estabelecimento.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dig_verificador: string (nullable = true)
 |-- idc_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cod_situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- data_inicio_atividade: string (nullable = true)
 |-- cnae_fiscal_principal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- nome_logradouro: string (nullable = true)
 |-- numero_logradouro: string (nullable = true)
 |-- complemento_logradouro: string (nullable = true)
 |-- nome_bairro: string (nullable = true)
 |-- cod_cep: string (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- cod_municipio: int

In [37]:
df_estabelecimento.write.mode("overwrite").parquet("gs://bootcamp-edc/trusted/estabelecimento.parquet")

In [38]:
del(df_estabelecimento)

In [39]:
spark.stop()